# **`Haleema Sadia ( B22F0543CS095 )`**
# **`Urooj Fatima  ( B22F0431CS021 )`**
# **`CS-Green`**
# **`AI LAB PROJECT:`** **`"Lane Line Dectection"`**  

  
# 🔍 Objective of the Project
The main objective of this project is to detect lane lines on roads using a video feed. This is a fundamental task in autonomous driving systems, where identifying road lanes helps vehicles stay in their lane and navigate properly.

# 🎯 Purpose of the Project
>Develop a simple lane detection system using image processing.

>Understand key computer vision techniques like edge detection, masking, and line detection.

>Practice real-time video processing with OpenCV.

# 🧠 Importance of Lane Detection
>Core component of self-driving cars – helps in path planning and avoiding accidents.

>Assists in driver safety systems – like lane departure warning systems.

>Applies computer vision concepts in a real-world scenario.
# 🧱 Step-by-Step Working of the Code
#### Step 1: Read Video Frame-by-Frame
                                          cap = cv2.VideoCapture(video_path)
>The program loads a video file to be processed.

>It then reads and processes each frame continuously inside a loop.

#### Step 2: Edge Detection using Canny
                                         def canny(img):
                                         ...
                                         gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                                         blur = cv2.GaussianBlur(gray, (kernel, kernel), 0)
                                         canny = cv2.Canny(blur, 50, 150)
                                         return canny

>The current video frame is first converted to grayscale to simplify the image and reduce complexity.

>A Gaussian blur is applied to smooth the image and remove noise.

>The Canny edge detector is then used to detect strong edges in the frame, such as lane markings.

#### Step 3: Apply Region of Interest (ROI)
                                         def region_of_interest(canny):
                                         ...
                                         triangle = np.array([[(200, height), (800, 350), (1200, height)]])
                                         ...
>A mask is created that focuses only on the lower part of the image where lanes are usually visible.

>This region is shaped like a triangle to simulate the area in front of a moving car.

>Only the edges within this region are kept; the rest are ignored.

#### Step 4: Detect Line Segments (Hough Transform)
                                        def hough_lines(cropped_canny):
                                        return cv2.HoughLinesP(...)
>A line detection technique called the Probabilistic Hough Transform is applied to detect straight line segments within the region of interest.

>It returns multiple short lines that possibly represent portions of lane lines.

#### Step 5: Average and Filter Detected Lines
                                       def average_slope_intercept(image, lines):
                                       ...
>The detected lines are categorized based on their slope:

>Lines with a negative slope are assumed to belong to the left lane.

>Lines with a positive slope are assumed to belong to the right lane.

>For both left and right lanes, the coordinates of all detected lines are averaged to form one single, clean line on each side.

>These averaged lines are then converted into start and end points to be drawn on the image.

                                      def make_points(image, line):
                                      ...
#### Step 6: Draw the Lane Lines
                                      def display_lines(img, lines):
                                      ...

>The averaged left and right lines are drawn using a bold colored line (commonly red) on a blank image.

>This image now clearly highlights where the lane lines are.

#### Step 7: Overlay Lane Lines on the Original Frame
                                     def add_weighted(frame, line_image):
                                     return cv2.addWeighted(frame, 0.8, line_image, 1, 1)
>The image with the lane lines is combined with the original video frame.

>This makes it easier to visualize the lanes directly over the driving scene.

#### Step 8: Display the Final Result in Real Time
                                    cv2.imshow("result", combo_image)
>The combined result (original frame + lane lines) is displayed in a window.

>The program continues updating this window frame-by-frame as the video plays.

>The display continues until a specific key (like 'q') is pressed to exit.

In [7]:
pip install opencv-python

In [2]:
pip install opencv-python-headless --user


  Using cached opencv_python_headless-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_python_headless-4.11.0.86-cp37-abi3-win_amd64.whl (39.4 MB)


In [12]:
import cv2
import numpy as np

def canny(img):
    if img is None:
        cap.release()
        cv2.destroyAllWindows()
        exit()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    kernel = 5
    blur = cv2.GaussianBlur(gray, (kernel, kernel), 0)
    canny = cv2.Canny(blur, 50, 150)
    return canny

def region_of_interest(canny):
    height = canny.shape[0]
    width = canny.shape[1]
    mask = np.zeros_like(canny)
    triangle = np.array([[
        (200, height),
        (800, 350),
        (1200, height),
    ]], np.int32)
    cv2.fillPoly(mask, triangle, 255)
    masked_image = cv2.bitwise_and(canny, mask)
    return masked_image

def hough_lines(cropped_canny):
    return cv2.HoughLinesP(cropped_canny, 2, np.pi/180, 100, np.array([]), minLineLength=40, maxLineGap=5)

def add_weighted(frame, line_image):
    return cv2.addWeighted(frame, 0.8, line_image, 1, 1)

def display_lines(img, lines):
    line_image = np.zeros_like(img)
    if lines is not None:
        for line in lines:
            for x1, y1, x2, y2 in line:
                cv2.line(line_image, (x1, y1), (x2, y2), (0, 0, 255), 10)
    return line_image

def make_points(image, line):
    slope, intercept = line
    y1 = int(image.shape[0])
    y2 = int(y1 * 3.0 / 5)
    x1 = int((y1 - intercept) / slope)
    x2 = int((y2 - intercept) / slope)
    return [[x1, y1, x2, y2]]

def average_slope_intercept(image, lines):
    left_fit = []
    right_fit = []
    if lines is None:
        return None
    for line in lines:
        for x1, y1, x2, y2 in line:
            fit = np.polyfit((x1, x2), (y1, y2), 1)
            slope = fit[0]
            intercept = fit[1]
            if slope < 0:
                left_fit.append((slope, intercept))
            else:
                right_fit.append((slope, intercept))

    averaged_lines = []

    if left_fit:
        left_fit_average = np.average(left_fit, axis=0)
        left_line = make_points(image, left_fit_average)
        averaged_lines.append(left_line)

    if right_fit:
        right_fit_average = np.average(right_fit, axis=0)
        right_line = make_points(image, right_fit_average)
        averaged_lines.append(right_line)

    return averaged_lines if averaged_lines else None

# Replace with your actual video path
video_path = r"test_video.mp4"
cap = cv2.VideoCapture(video_path)

while cap.isOpened():
    _, frame = cap.read()
    if frame is None:
        break

    canny_image = canny(frame)
    cropped_canny = region_of_interest(canny_image)

    lines = hough_lines(cropped_canny)
    averaged_lines = average_slope_intercept(frame, lines)

    line_image = display_lines(frame, averaged_lines) if averaged_lines is not None else np.zeros_like(frame)
    combo_image = add_weighted(frame, line_image)
    
    cv2.imshow("result", combo_image)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# 🧾 Summary of Code Components

Function	              |Purpose
|---------------|--------------------------------------------------|
canny()	                  |Preprocess frame to detect edges.
region_of_interest()	  |Focus on road area only.
hough_lines()	          |Detect potential lane lines.
average_slope_intercept() |Clean and combine multiple lines.
display_lines()	          |Draw detected lines.
add_weighted()	          |Overlay lines onto original image

# 🧠 Concepts Applied
>Edge Detection

>Region Masking

>Line Detection (Hough Transform)

>Slope Calculation and Line Fitting

>Video Processing and Real-time Visualization
# 🧱 Description
The video is read frame-by-frame for lane detection.

Canny edge detection is applied to identify lane edges.

A region of interest (ROI) is defined to focus on the road area.

The Hough Transform detects and averages the lane lines.

The final lane lines are overlaid on the original video and displayed in real-time.
# ✅ Final Output
You will see a video where the left and right lane lines are detected and highlighted in red as the car moves forward.